# Using Numpy only

## Linear Regression

In [1]:
import numpy as np

class LinearRegression():
    def __init__(self, eps=1e-6):
        self.eps = eps

    def fit(self, x, y, lr=0.01, add_intercept=True):
        if add_intercept:
            x = np.hstack([np.ones((x.shape[0], 1)), x])
        m, n = x.shape

        self.theta = np.zeros(n)

        while True:
            theta_old = self.theta.copy()
            h_x = x.dot(self.theta)
            grad = (x.T.dot(h_x - y)) / m
            self.theta -= lr * grad

            if np.linalg.norm(self.theta - theta_old, ord=1) < self.eps:
                break

    def predict(self, x, add_intercept=True):
        if add_intercept:
            x = np.hstack([np.ones((x.shape[0], 1)), x])
        return x.dot(self.theta)

## Logistic Regression

In [2]:
import numpy as np

class LogisticRegression():
    def __init__(self, eps=1e-6):
        self.eps = eps

    def fit(self, x, y, add_intercept=True, lr=0.01, max_iter=10000):
        if add_intercept:
            x = np.hstack([np.ones((x.shape[0], 1)), x])

        m, n = x.shape
        self.theta = np.zeros(n)

        for _ in range(max_iter):
            theta_old = self.theta.copy()
            z = np.clip(x.dot(self.theta), -500, 500)
            h_x = 1 / (1 + np.exp(-z))
            grad = (x.T.dot(h_x - y)) / m
            self.theta -= lr * grad

            if np.linalg.norm(self.theta - theta_old, ord=1) < self.eps:
                break

    def predict_proba(self, x, add_intercept=True):
        if add_intercept:
            x = np.hstack([np.ones((x.shape[0], 1)), x])
        z = np.clip(x.dot(self.theta), -500, 500)
        return 1 / (1 + np.exp(-z))

    def predict(self, x, add_intercept=True):
        return (self.predict_proba(x, add_intercept) >= 0.5).astype(int)


# Using Pytorch

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

## Linear regression

In [4]:
torch.manual_seed(0)

x = torch.linspace(0, 10, 100).view(-1, 1)  
y = 2 * x + 3 + torch.randn_like(x) 

x_numpy = x.numpy()
y_numpy = y.numpy()

In [5]:
class LinearRegression_pytorch(nn.Module):
    def __init__(self):
        super(LinearRegression_pytorch, self).__init__()
        self.linear = nn.Linear(1,1)
    def forward(self, x):
        return self.linear(x)
    
model = LinearRegression_pytorch()
criterion = nn.MSELoss()
optimiser = optim.SGD(model.parameters(), lr=0.01)

epochs = 1000
losses = []

for epoch in range(epochs):
    y_pred = model(x)
    loss = criterion(y_pred, y)
    optimiser.zero_grad()
    loss.backward()
    optimiser.step()

    losses.append(loss.item())
    if epoch%100 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [1/1000], Loss: 246.7126
Epoch [101/1000], Loss: 1.6820
Epoch [201/1000], Loss: 1.2833
Epoch [301/1000], Loss: 1.1359
Epoch [401/1000], Loss: 1.0814
Epoch [501/1000], Loss: 1.0613
Epoch [601/1000], Loss: 1.0538
Epoch [701/1000], Loss: 1.0511
Epoch [801/1000], Loss: 1.0501
Epoch [901/1000], Loss: 1.0497


## Logistic regression

In [6]:
class LogisticRegression_pytorch(nn.Module):
    def __init__(self, input_dim, output_dim=1):
        super(LogisticRegression_pytorch, self).__init__()
        self.linear = nn.Linear(output_dim,input_dim)
    def forward(self, x):
        return torch.sigmoid(self.linear(x))

model = LogisticRegression_pytorch(input_dim=x.shape[1])
criterion = nn.BCELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

## now use training loop


## GDA

In [7]:
import numpy as np

class GDA:
    def __init__(self, eps=1e-16):
        self.eps = eps
        self.phi = None
        self.mu0 = None
        self.mu1 = None
        self.sigma = None

    def fit(self, X, Y):
        m, n = X.shape
        Y = Y.flatten()

        count_y1 = np.sum(Y == 1)
        count_y0 = m - count_y1
        
        self.phi = count_y1 / m
        
        X0 = X[Y == 0]
        X1 = X[Y == 1]
        
        self.mu0 = np.mean(X0, axis=0)
        self.mu1 = np.mean(X1, axis=0)

        diff = np.zeros_like(X)
        diff[Y == 0] = X0 - self.mu0
        diff[Y == 1] = X1 - self.mu1
        
        self.sigma = (diff.T @ diff) / m

    def predict(self, X):
        log_prob0 = self.log_prob(X, self.mu0, self.sigma) + np.log(1 - self.phi + self.eps)
        log_prob1 = self.log_prob(X, self.mu1, self.sigma) + np.log(self.phi + self.eps)
        
        predictions = (log_prob1 > log_prob0).astype(int)
        
        return predictions

    def log_prob(self, X, mu, sigma):
        n_features = X.shape[1]
        
        inv_sigma = np.linalg.inv(sigma + np.eye(n_features) * self.eps)
        det_sigma = np.linalg.det(sigma)
        
        log_const = -0.5 * n_features * np.log(2 * np.pi) - 0.5 * np.log(det_sigma + self.eps)
        diff = X - mu
        log_exp = -0.5 * np.sum(diff @ inv_sigma * diff, axis=1)
        
        return log_const + log_exp

## Naive Bayes for text classification
It will be use ful when we want to predict spam and non spam from the given messages.

In [8]:
import numpy as np

class naive_baiyes():
    def __init__(self):
        pass

    def fit(self, matrix, labels):
        n, V = matrix.shape
        y = labels
        self.phi_y_1 = np.sum(y==1)/n
        phi_k_given_0 = np.zeros((V,))
        phi_k_given_1 = np.zeros((V,))
        for i in range(n):
            for j in range(V):
                if y[i]==0:
                    phi_k_given_0[j]+= (matrix[i][j])
                else:
                    phi_k_given_1[j]+= (matrix[i][j])

        self.phi_k_given_1 = (phi_k_given_1 +1)/(np.sum(phi_k_given_1)+V)
        self.phi_k_given_0 = (phi_k_given_0 +1)/(np.sum(phi_k_given_0)+V)
    
    def predict(self,matrix ):
        sum_log_p_x_y1 = (np.log(self.phi_k_y1) * matrix).sum(axis=1) + np.log(self.phi_y)
        sum_log_p_x_y0 = (np.log(self.phi_k_y0) * matrix).sum(axis=1) + np.log(1 - self.phi_y)
        return (sum_log_p_x_y1 > sum_log_p_x_y0).astype(int)